In [1]:
# Importing Libraries/Dependencies
import selenium
import pandas as pd
import time
import requests
from selenium import webdriver
from  bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException

## Scraping Reviews from https://www.flipkart.com/

In [2]:
# Connecting to the webdriver 
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window()        # Maximizing the window

In [3]:
# Getting the webpage of mentioned url
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(2)
# Clicking on cancel button 
driver.find_element_by_xpath("//div[@class='_2QfC02']/button").click()

In [4]:
# Listing down the items that need to be searched
srch_items = ['laptops', 'Phones','smart watches','Monitors']

In [5]:
# Creating empty lists to fetch required data
title = []           # Title of the product
review_text = []     # Text of the review or comments of the particular product
ratings = []         # Rating of the product

In [6]:
# Defining function to scrap the required data of products
def scrap():    
        for i in driver.find_elements_by_xpath("//div[@class='t-ZTKy']"):
            review_text.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']"):
            title.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']/../div"):
            ratings.append(i.text)
        return

In [7]:
# Creating empty list to fetch the page URLs of the products
urls=[]

In [8]:
for i in srch_items:
    # Find search bar
    srchBar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input")
    srchBar.clear()
    srchBar.send_keys(i)
    # Clicking on search button
    driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
    time.sleep(3)
    page = []
    for i in driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a"):
        page.append(i.get_attribute('href'))
    for i in page[0:4]:
        driver.get(i)
        time.sleep(3)
        items = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
        for i in items:
            urls.append(i.get_attribute('href'))

In [9]:
# Checking the length of the url
len(urls)

384

In [10]:
# Fetching the required data from the website
for i in urls:
    driver.get(i)
    time.sleep(2)
    for _ in range(2):
        driver.execute_script("window.scrollBy(0,6000)")
        time.sleep(1)
        # Clicking on all reviews
    try:
        btn=driver.find_element_by_xpath("//div[@class='_2c2kV-']/following::a")
        lnk = btn.get_attribute('href')
        driver.get(lnk)
        time.sleep(1)
    except NoSuchElementException:
        pass
    
# Calling the function
    scrap()        
    try:
        n_page=driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a")
        np=[]
        for i in n_page:
            np.append(i.get_attribute('href'))
        for i in np[0:18]:
            driver.get(i)
            time.sleep(1)
            scrap()
    except: continue

In [11]:
# Checking the length of the columns
len(ratings), len(review_text), len(title)

(28431, 28431, 28431)

In [12]:
# Creating the dataframe for scraped data from the flipkart website
data = list(zip(title,review_text,ratings))
df2 = pd.DataFrame(data, columns = ["Review_title","Reiew_text","Ratings"])
df2

,Review_title,Reiew_text,Ratings
0,Brilliant,It is good for online classes and for slightly...,5
1,Brilliant,Superb,5
2,Terrific purchase,Very good,5
3,Worth every penny,All good,5
4,Highly recommended,"Very nice and good finishing, light weight, go...",5
...,...,...,...
28426,Fair,Goods:\nThe picture quality is good.\nHeight a...,3
28427,Wonderful,Too much of back light bleeding.\nWorth the mo...,4
28428,Design looks very good,Design looks very good. fixed stand so no adju...,5
28429,Brilliant,I love product. Very nice working.,5


In [13]:
# Saving the scraped data into csv file
df2.to_csv("Flipkrt_reviews.csv")

## Scraping Reviews from https://www.amazon.in/

In [15]:
# Connecting to the webdriver 
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window()        # Maximizing the window

In [16]:
# Getting the webpage of mentioned url
url = "https://www.amazon.in/"
driver.get(url)
time.sleep(3)

In [17]:
# Listing down the items that need to be searched
srch_items = ['laptops','phones','headphones','smart watches', 'Professional Cameras', 'Printers', 'Monitors', 'Home theater', 'Router']

In [18]:
# Creating empty lists to fetch required data
title = []           # Title of the product
review_text = []     # Text of the review or comments of the particular product
ratings = []         # Rating of the product

In [19]:
# Fetching the required data from the website
for i in srch_items:
    srchbar = driver.find_element_by_id("twotabsearchtextbox")
    srchbar.send_keys(i)
    driver.find_element_by_id("nav-search-submit-button").click()  # Clicking on search button
    time.sleep(1)
    item_url = []
    for i in driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a"):
        item_url.append(i.get_attribute('href'))
    for i in item_url:
        driver.get(i)
        time.sleep(1)
        for _ in range(2):
            driver.execute_script("window.scrollBy(0,6000)")
            time.sleep(1)
        # Clicking on see all reviews
        try:
            driver.find_element_by_xpath("//a[@class='a-link-emphasis a-text-bold']").click()
        except:
            pass
        
        url_ = []
        try:
            two_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[4]/td[2]/a")
            url_.append(two_str.get_attribute('href'))
        except: pass
        try:
            three_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[3]/td[2]/a")
            url_.append(three_str.get_attribute('href'))
        except: pass
        try:
            one_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[5]/td[2]/a")
            url_.append(one_str.get_attribute('href'))
        except: pass
        try:
            fiv_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[1]/td[2]/a")
            url_.append(fiv_str.get_attribute('href'))
        except: pass
        try:
            four_str = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[2]/td[2]/a")
            url_.append(four_str.get_attribute('href'))
        except: pass
        
        for i in url_:
            driver.get(i)
            time.sleep(1)
            for i in range(10):
                ids = driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none review-views celwidget']/div")
                comment_ids = []
                for i in ids:
                    comment_ids.append(i.get_attribute('id'))

                for x in comment_ids:
                    # Extract user ids from each user on a page
                    try:
                        review_title = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[2]/a[2]/span[1]')
                        title.append(review_title.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        title.append('')

                    try:
                        rating = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[2]/a[1]/i/span[1]')
                        ratings.append(rating.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        ratings.append('')

                    try:
                        text = driver.find_element_by_xpath('//*[@id="' + x +'"]/div/div/div[4]/span/span')
                        review_text.append(text.get_attribute("innerHTML"))
                    except NoSuchElementException:
                        review_text.append('')
                try:
                    driver.find_element_by_xpath("//ul[@class='a-pagination']/li[2]/a").click()
                    time.sleep(3)
                except : break 


In [20]:
len(ratings), len(title), len(review_text)

(72697, 72697, 72697)

In [21]:
#creating a dataframe
data = list(zip(title,review_text,ratings))
df = pd.DataFrame(data, columns = ["Review_title","Reiew_text","Ratings"])
df

,Review_title,Reiew_text,Ratings
0,Not suitable for work,\n It took me one month to get my laptop work...,2.0 out of 5 stars
1,Not worth for what you pay,\n You can buy the same for better price dire...,2.0 out of 5 stars
2,Good Product,\n Overall Product... upgraded windows 11 and...,2.0 out of 5 stars
3,Not advised,\n Theres a scratch already in the screen and...,2.0 out of 5 stars
4,"The laptop hangs, sometimes gets stuck and it ...",\n The laptop is slow sometimes get stuck and...,2.0 out of 5 stars
...,...,...,...
72692,Vfm. Range of Archer c6 or A6 without mu-mimo,\n Same wifi signal range as c6 or the new A6...,4.0 out of 5 stars
72693,USB port is not supported for printer,\n USB port is not supporting printer so I a...,4.0 out of 5 stars
72694,A best buy for that price range,"\n Like : Speed , look and also better range ...",4.0 out of 5 stars
72695,Value For Money,\n Got it for 2K. Wifi signal in 2.4 Ghz is v...,4.0 out of 5 stars


In [22]:
# Saving the data into csv file
df.to_csv("Amazon_reviews.csv")

In [23]:
# lets load the flipkart reviews file
flpkrt = pd.read_csv("Flipkrt_reviews.csv")
flpkrt

,Unnamed: 0,Review_title,Reiew_text,Ratings
0,0,Brilliant,It is good for online classes and for slightly...,5
1,1,Brilliant,Superb,5
2,2,Terrific purchase,Very good,5
3,3,Worth every penny,All good,5
4,4,Highly recommended,"Very nice and good finishing, light weight, go...",5
...,...,...,...,...
28426,28426,Fair,Goods:\nThe picture quality is good.\nHeight a...,3
28427,28427,Wonderful,Too much of back light bleeding.\nWorth the mo...,4
28428,28428,Design looks very good,Design looks very good. fixed stand so no adju...,5
28429,28429,Brilliant,I love product. Very nice working.,5


In [24]:
# Concatenating the two datasets 
reviews = pd.concat([df, flpkrt],ignore_index=True)
reviews

,Review_title,Reiew_text,Ratings,Unnamed: 0
0,Not suitable for work,\n It took me one month to get my laptop work...,2.0 out of 5 stars,NaN
1,Not worth for what you pay,\n You can buy the same for better price dire...,2.0 out of 5 stars,NaN
2,Good Product,\n Overall Product... upgraded windows 11 and...,2.0 out of 5 stars,NaN
3,Not advised,\n Theres a scratch already in the screen and...,2.0 out of 5 stars,NaN
4,"The laptop hangs, sometimes gets stuck and it ...",\n The laptop is slow sometimes get stuck and...,2.0 out of 5 stars,NaN
...,...,...,...,...
101123,Fair,Goods:\nThe picture quality is good.\nHeight a...,3,28426.0
101124,Wonderful,Too much of back light bleeding.\nWorth the mo...,4,28427.0
101125,Design looks very good,Design looks very good. fixed stand so no adju...,5,28428.0
101126,Brilliant,I love product. Very nice working.,5,28429.0


In [25]:
# Dropping unwanted column
reviews.drop(columns = 'Unnamed: 0', inplace=True)

In [26]:
# Displaying the merged dataset
reviews

,Review_title,Reiew_text,Ratings
0,Not suitable for work,\n It took me one month to get my laptop work...,2.0 out of 5 stars
1,Not worth for what you pay,\n You can buy the same for better price dire...,2.0 out of 5 stars
2,Good Product,\n Overall Product... upgraded windows 11 and...,2.0 out of 5 stars
3,Not advised,\n Theres a scratch already in the screen and...,2.0 out of 5 stars
4,"The laptop hangs, sometimes gets stuck and it ...",\n The laptop is slow sometimes get stuck and...,2.0 out of 5 stars
...,...,...,...
101123,Fair,Goods:\nThe picture quality is good.\nHeight a...,3
101124,Wonderful,Too much of back light bleeding.\nWorth the mo...,4
101125,Design looks very good,Design looks very good. fixed stand so no adju...,5
101126,Brilliant,I love product. Very nice working.,5


In [29]:
# Saving the data into csv file
reviews.to_csv("Review&Rating.csv",index=False)

In [30]:
# Saving the data into excel file
reviews.to_excel("Review&Ratings.xlsx",index=False)